# Selecting target sources from GAIA DR2
### https://ui.adsabs.harvard.edu/abs/2018A%26A...620A.128V/abstract

In [1]:
#### Conda environment
## environment terada2019
# conda create --name terada2019 python=3.7
# pip install --upgrade pip
# pip install astropy scipy
# pip install photutils
# pip install jupyter matplotlib h5py aplpy pyregion PyAVM healpy
# pip install astroquery
# pip install pandas
# pip install -U statsmodels
# pip install -U scikit-learn

In [2]:
import os
import sys
import time
sys.path
sys.path.append('./')

from astropy.io import fits

In [6]:
!ls

asu.fits  gaiaHerbig.ipynb


In [19]:
# Loading FITS table
inFITS = 'asu.fits'
hdul = fits.open(inFITS)

hdul.fileinfo(0)

print( "number of tables: ", len(hdul) )

number of tables:  2


In [31]:
# print headers

for index in range(0, len(hdul) ):
    print( '#############   TABLE ' + str(index) + '   ##############' )
    hdr = hdul[index].header
    print( repr(hdr) )

#############   TABLE 0   ##############
SIMPLE  =                    T / Standard FITS Format                           
BITPIX  =                    8 / Character data                                 
NAXIS   =                    0 / No Image --- just extension(s)                 
EXTEND  =                    T / There are standard extensions                  
ORIGIN  = 'xml2fits_v1.95'     / Converted from XML-Astrores to FITS            
                         e-mail: question@simbad.u-strasbg.fr                   
COMMENT  ARG='-rm'                                                              
LONGSTRN= 'OGIP 1.0'           / Long string convention (&/CONTINUE) may be used
DATE    = '2020-03-19'         / Written on 2020-03-19:04:48:06 (GMT)           
                             by: apache@vizier.u-strasbg.fr                     
         **********************************************************             
             EXCERPT from catalogues stored in VizieR (CDS)         

In [48]:
# list keys
datatab_id = 1
list( hdul[datatab_id].header.keys() )

['XTENSION',
 'BITPIX',
 'NAXIS',
 'NAXIS1',
 'NAXIS2',
 'PCOUNT',
 'GCOUNT',
 'TFIELDS',
 'CDS-CAT',
 '',
 'EXTNAME',
 'CDS-NAME',
 '',
 '',
 'UCD__1',
 'TDISP1',
 'TFORM1',
 'TTYPE1',
 'UCD__2',
 'TDISP2',
 'TFORM2',
 'TTYPE2',
 '',
 'TUNIT2',
 'UCD__3',
 'TDISP3',
 'TFORM3',
 'TTYPE3',
 '',
 'TUNIT3',
 'UCD__4',
 'TDISP4',
 'TFORM4',
 'TTYPE4',
 'TUNIT4',
 'UCD__5',
 'TDISP5',
 'TFORM5',
 'TTYPE5',
 '',
 'TUNIT5',
 'UCD__6',
 'TDISP6',
 'TFORM6',
 'TTYPE6',
 'TUNIT6',
 'UCD__7',
 'TDISP7',
 'TFORM7',
 'TTYPE7',
 'TUNIT7',
 'UCD__8',
 'TDISP8',
 'TFORM8',
 'TTYPE8',
 'TUNIT8',
 'UCD__9',
 'TDISP9',
 'TFORM9',
 'TTYPE9',
 'TUNIT9',
 'UCD__10',
 'TDISP10',
 'TFORM10',
 'TTYPE10',
 '',
 'UCD__11',
 'TDISP11',
 'TFORM11',
 'TTYPE11',
 '',
 'UCD__12',
 'TDISP12',
 'TFORM12',
 'TTYPE12',
 '',
 'UCD__13',
 'TDISP13',
 'TFORM13',
 'TTYPE13',
 '',
 'TUNIT13',
 'UCD__14',
 'TDISP14',
 'TFORM14',
 'TTYPE14',
 '',
 'UCD__15',
 'TDISP15',
 'TFORM15',
 'TTYPE15',
 '',
 'UCD__16',
 'TDISP16',
 'TFO

In [51]:
# loading data into python structures
datatab_id = 1
num_AllSC = hdul[datatab_id].header['naxis2'] 

data = hdul[datatab_id].data
data.shape

# Listing available columns
hdul[1].columns

#print('###########')
#print( hdul[1].columns.names )

ColDefs(
    name = 'Name'; format = '13A'; disp = 'A13'
    name = 'RAICRS'; format = 'D'; unit = 'deg'; disp = 'F10.4'
    name = 'DEICRS'; format = 'D'; unit = 'deg'; disp = 'F9.4'
    name = 'plx'; format = 'D'; unit = 'mas'; disp = 'F9.6'
    name = 'Dist'; format = 'E'; unit = 'pc'; disp = 'F6.1'
    name = 'Teff'; format = 'J'; unit = 'K'; disp = 'I5'
    name = 'LogL'; format = 'E'; unit = '[Lsun]'; disp = 'F5.2'
    name = 'Av'; format = 'E'; unit = 'mag'; disp = 'F6.3'
    name = 'V'; format = 'E'; unit = 'mag'; disp = 'F5.2'
    name = 'Bin'; format = '3A'; disp = 'A3'
    name = 'E_NIR_'; format = 'E'; disp = 'E10.3'
    name = 'E_MIR_'; format = 'E'; disp = 'E10.3'
    name = 'EWHa'; format = 'D'; unit = '0.1nm'; disp = 'F8.3'
    name = 'Hashape'; format = 'A'; disp = 'A1'
    name = 'Vi'; format = 'E'; disp = 'F5.2'
    name = 'UXOR'; format = '3A'; disp = 'A3'
    name = 'Mass'; format = 'E'; unit = 'Msun'; disp = 'F6.3'
    name = 'Age'; format = 'E'; unit = 'Myr'; dis